In [1]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: C:\Users\singh\desktop\pythonn\llm\cuda\Scripts\python.exe -m pip install --upgrade pip


In [2]:
from datasets  import load_dataset

C:\Users\singh\desktop\pythonn\llm\cuda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
raw=load_dataset("glue",'sst2')

In [4]:
import numpy as np
from pprint import pprint as pp

In [5]:
raw['train'][0]

{'sentence': 'hide new secretions from the parental units ',
 'label': 0,
 'idx': 0}

In [41]:
from transformers import AutoTokenizer as at

In [42]:
checkpoint="bert-base-uncased"
tokenizer=at.from_pretrained(checkpoint)

In [43]:
tokenized_sentences=tokenizer(raw['train'][0:3]['sentence'],padding=True,truncation=True)

In [44]:
tokens=tokenizer.tokenize(raw['train'][0:4]['sentence'])
tokens

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [ ]:
(raw['train'][0:4]['sentence']).map()

In [45]:
def tokeniztion(batch):
    return tokenizer(batch['sentence'],truncation=True,padding=True)

In [46]:
tokenizd_datset=raw.map(tokeniztion,batched=True)

Map: 100%|█████████████████████████████████████████████████████████████| 67349/67349 [00:04<00:00, 15685.97 examples/s]


In [ ]:
tokenizd_datset["train"]['input_ids']

In [38]:
import transformers

In [57]:
ta=transformers.TrainingArguments(
    "My trainer",
save_strategy="epoch",
num_train_epochs=1,
    evaluation_strategy="epoch"
    
)

In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
model=AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=2)

In [ ]:
from torchinfo import summary

In [ ]:
summary(model)

In [ ]:
from transformers import Trainer

In [ ]:
from datasets import load_metric

In [ ]:
metric=load_metric("glue","sst2")

In [60]:
def compute_metrics(logits_and_labels):
    logits, labels = logits_and_labels
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [61]:
trainer=Trainer(model,ta,train_dataset=tokenizd_datset["train"],eval_dataset=tokenizd_datset["validation"],tokenizer=tokenizer,compute_metrics=compute_metrics)

In [62]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.182200,0.344894,0.915138


TrainOutput(global_step=8419, training_loss=0.17650228079657526, metrics={'train_runtime': 1771.4522, 'train_samples_per_second': 38.019, 'train_steps_per_second': 4.753, 'total_flos': 2189113786144440.0, 'train_loss': 0.17650228079657526, 'epoch': 1.0})

In [63]:
trainer.save_model("First_big")

In [65]:
from transformers import pipeline

In [66]:
newmodel=pipeline("text-classification",model="First_big",device=0)

In [71]:
newmodel("I had a disappointing experience with this product. The quality fell short of my expectations, and I encountered several issues during usage. The user interface is confusing, and the overall performance is subpar. I wouldn't recommend it to others based on my experience.")

[{'label': 'LABEL_0', 'score': 0.9916580319404602}]